# Qiskit Multirunner

### Imports

In [28]:
import quimb.tensor
import qiskit
from qiskit import QuantumCircuit, QuantumRegister
import qiskit_aer
from mqt import ddsim

from qiskit_aer import AerSimulator
from qiskit_quimb import quimb_circuit
from qiskit_qulacs import QulacsProvider

### Sample Circuit

In [ ]:
def get_sample_circuit() -> QuantumCircuit:
    qubits = QuantumRegister(1)
    qc = QuantumCircuit(qubits)
    qc.h(qubits[0])
    qc.measure_all()
    return qc

In [30]:
# Display our sample circuit
qc = get_sample_circuit()
qc.draw()

┌───┐
q13: ┤ H ├
     └───┘

### Qiskit AER Execution

In [31]:
def run_on_aer(qc: QuantumCircuit):
    backend = AerSimulator()
    result = backend.run(qc).result()
    return result
run_on_aer(get_sample_circuit())

Result(backend_name='aer_simulator', backend_version='0.16.3', qobj_id='', job_id='d4b1b728-8a5b-4ac9-bbb1-d559bfc9d729', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(creg_sizes=[], global_phase=0.0, memory_slots=0, n_qubits=1, name='circuit-175', qreg_sizes=[['q14', 1]], metadata={}), status=DONE, seed_simulator=3142879630, metadata={'time_taken': 4.4744e-05, 'num_bind_params': 1, 'required_memory_mb': 0, 'input_qubit_map': [], 'method': 'stabilizer', 'device': 'CPU', 'num_qubits': 0, 'parallel_state_update': 8, 'active_input_qubits': [], 'num_clbits': 0, 'remapped_qubits': False, 'parallel_shots': 1, 'runtime_parameter_bind': False, 'max_memory_mb': 15900, 'measure_sampling': False, 'batched_shots_optimization': False}, time_taken=4.4744e-05)], date=2025-03-27T18:51:09.634734, status=COMPLETED, header=None, metadata={'time_taken_parameter_binding': 5.49e-06, 'time_taken_execute': 9.2125e-05, '

### Qiskit MQT Execution

In [32]:
def run_on_mqt(qc: QuantumCircuit):
    backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")
    result = backend.run(qc).result()
    return result

run_on_mqt(get_sample_circuit())

Result(backend_name='qasm_simulator', backend_version='1.24.0', qobj_id='None', job_id='9cb80df0-3e19-4167-aa7a-ec8ae2b349c7', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 536, '0x1': 488}, time_taken=0.008974790573120117), header=DDSIMHeader(name='circuit-176', n_qubits=1, memory_slots=0, global_phase=0.0, creg_sizes=[], clbit_labels=[], qreg_sizes=[('q15', 1)], qubit_labels=[('q15', 0)]), status=DONE, seed=-1, metadata={})], date=None, status=COMPLETED, header=None, time_taken=0.009807825088500977)

### Qiskit Qulacs Execution

In [33]:
def run_on_qulacs(qc: QuantumCircuit):
    backend = QulacsProvider().get_backend('qulacs_simulator')
    result = backend.run(qc).result()
    return result

run_on_qulacs(get_sample_circuit())

/tmp/ipykernel_3016/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')


Result(backend_name='qulacs_simulator', backend_version='0.1.0', qobj_id='0', job_id='070238d6-e17c-4329-ba29-dfb70b189f77', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(statevector=[0.70710678+0.j 0.70710678+0.j]), status=JobStatus.DONE)], date=None, status=JobStatus.DONE, header=None, time_taken=6.651878356933594e-05)

In [37]:
def log_1000_simulations_per_sumulator():
    circuit = get_sample_circuit()
    for i in range(0,999):
        print(run_on_aer(circuit))
    for i in range(0,999):
        print(run_on_mqt(circuit))
    for i in range(0,999):
        print(run_on_qulacs(circuit))
log_1000_simulations_per_sumulator()

Result(backend_name='aer_simulator', backend_version='0.16.3', qobj_id='', job_id='d7f9ca36-9cc5-4c84-b4e9-b31303709b96', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(creg_sizes=[], global_phase=0.0, memory_slots=0, n_qubits=1, name='circuit-179', qreg_sizes=[['q18', 1]], metadata={}), status=DONE, seed_simulator=3959470547, metadata={'time_taken': 4.1072e-05, 'num_bind_params': 1, 'required_memory_mb': 0, 'input_qubit_map': [], 'method': 'stabilizer', 'device': 'CPU', 'num_qubits': 0, 'parallel_state_update': 8, 'active_input_qubits': [], 'num_clbits': 0, 'remapped_qubits': False, 'parallel_shots': 1, 'runtime_parameter_bind': False, 'max_memory_mb': 15900, 'measure_sampling': False, 'batched_shots_optimization': False}, time_taken=4.1072e-05)], date=2025-03-27T18:58:33.643206, status=COMPLETED, header=None, metadata={'time_taken_parameter_binding': 5.917e-06, 'time_taken_execute': 8.9336e-05, 

/tmp/ipykernel_3016/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')


Result(backend_name='qulacs_simulator', backend_version='0.1.0', qobj_id='0', job_id='ee7eb307-cd48-43bb-a385-d3d6495b74ab', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(statevector=[0.70710678+0.j 0.70710678+0.j]), status=JobStatus.DONE)], date=None, status=JobStatus.DONE, header=None, time_taken=2.4080276489257812e-05)
Result(backend_name='qulacs_simulator', backend_version='0.1.0', qobj_id='0', job_id='21af38e0-1d73-4830-b183-2a827142ad82', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(statevector=[0.70710678+0.j 0.70710678+0.j]), status=JobStatus.DONE)], date=None, status=JobStatus.DONE, header=None, time_taken=3.457069396972656e-05)
Result(backend_name='qulacs_simulator', backend_version='0.1.0', qobj_id='0', job_id='a84dee5d-866e-42ee-8c77-170dbd16716f', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(statevector